In [5]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [6]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
220,I saw this movie in Blockbuster and thought it...,negative
462,"Nominated for the oscar ""worst script ever"" in...",negative
595,"In a future society, the military component do...",positive
216,"A cheesy ""B"" crime thriller of the early '50, ...",negative
748,"OK, Chuck Norris has shown up in many an enter...",negative


In [10]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...


True

In [11]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [12]:
df = normalize_text(df)
df.head()

,review,sentiment
220,saw movie blockbuster thought might good sunda...,negative
462,nominated oscar worst script ever opinion dece...,negative
595,future society military component recruit rath...,positive
216,cheesy b crime thriller early story droll char...,negative
748,ok chuck norris shown many entertaining movie ...,negative


In [13]:
df['sentiment'].value_counts()

sentiment
negative    262
positive    238
Name: count, dtype: int64

In [14]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [15]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
220,saw movie blockbuster thought might good sunda...,0
462,nominated oscar worst script ever opinion dece...,0
595,future society military component recruit rath...,1
216,cheesy b crime thriller early story droll char...,0
748,ok chuck norris shown many entertaining movie ...,0


In [16]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [17]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [20]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/harsh392/Capstone-project.mlflow')
dagshub.init(repo_owner='harsh', repo_name='CAPSTONE-proj', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Repository CAPSTONE-proj doesn't exist, creating it under current user.

Initialized MLflow to track repo "harsh/CAPSTONE-proj"

Repository harsh/CAPSTONE-proj initialized!

2026/01/15 20:47:38 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/96416936898f4a5c9ee78a675c46334f', creation_time=1768490252903, experiment_id='0', last_update_time=1768490252903, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [21]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2026-01-15 20:47:39,133 - INFO - Starting MLflow run...
2026-01-15 20:47:39,768 - INFO - Logging preprocessing parameters...
2026-01-15 20:47:41,074 - INFO - Initializing Logistic Regression model...
2026-01-15 20:47:41,075 - INFO - Fitting the model...
2026-01-15 20:47:41,117 - INFO - Model training complete.
2026-01-15 20:47:41,118 - INFO - Logging model parameters...
2026-01-15 20:47:41,472 - INFO - Making predictions...
2026-01-15 20:47:41,474 - INFO - Calculating evaluation metrics...
2026-01-15 20:47:41,489 - INFO - Logging evaluation metrics...
2026-01-15 20:47:43,051 - INFO - Saving and logging the model...
2026/01/15 20:47:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026-01-15 20:48:00,787 - INFO - Model training and logging completed in 21.02 seconds.
2026-01-15 20:48:00,788 - INFO - Accuracy: 0.664
2026-01-15 20:48:00,789 - INFO - Precision: 0.6129032258064516
2026-01-15 20:48:00,789 - INFO - Recall: 0.6785714285714286
2026-01-15

🏃 View run gaudy-eel-151 at: https://dagshub.com/harsh392/Capstone-project.mlflow/#/experiments/0/runs/c9b6c7086fdd4a34a871f4a823c76341
🧪 View experiment at: https://dagshub.com/harsh392/Capstone-project.mlflow/#/experiments/0
